## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
!/databricks/python3/bin/python -m pip install --upgrade pip

In [0]:
# create a test directory in the /tmp/labuser104917-3007322/test

# copy 10 files 

# do dbutils.fs.mkdir
# do dbutils.cp to copy the 10 files 

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
# ENTER YOUR CODE HERE

# how to align your partitions with the number of clusters you are working with 
# we have 4 cores in this cluster so 4 partitions 

spark.conf.set("spark.sql.shuffle.partitions", "4")  # we have 4 cores in this cluster so 4 partitions 
spark.conf.set("spark.sql.adaptive.enabled", "true")

# NEED to have the necessary libraries 
import pandas as pd
import matplotlib # visualization 
import plotly # for graphing 
import json
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time
from pyspark.sql.functions import col, isNull, when 


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# ENTER YOUR CODE HERE

# Checking the contents of the TWEET_SOURCE_PATH directory and displaying it
df = get_source_listing_df()
display(df)



In [0]:
# Counting the number of files in the TWEET_SOURCE_PATH directory
count_of_files = df.count()
print("Number of source files:", count_of_files)

# print the contents of a singular file 
show_s3_file_contents("voc_volume/0.json")

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
from pyspark.sql.types import LongType, StringType, StructType, StructField, DateType

# raw schema
jsonschema_raw = StructType([
    StructField("date", StringType()),
    StructField("user", StringType()),
    StructField("text", StringType()),
    StructField("sentiment", StringType()),
])

from pyspark.sql.functions import col, current_timestamp

# cloud files to read the stream
bronze_query = (spark.readStream
    .format("cloudFiles")  # tells Spark to use AutoLoader
    .option("cloudFiles.format", "json")  # the actual format of our data files. Tells AutoLoader to expect json files
    .option("path", TWEET_SOURCE_PATH)  # where the tweets are coming from 
    .option("mergeSchema", True)  # ensuring mergeSchema works
    .schema(jsonschema_raw)  # raw schema of the data 
    .load()
    .withColumn('source_file', input_file_name())                # adding source file column               
    .withColumn('processing_time', current_timestamp()) # adding the time stamp of processing 
    .writeStream
    .format("delta")
    .outputMode("append")
    .queryName('bronze_stream')
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .start(BRONZE_DELTA)
)

bronze_query.awaitTermination(60) # lets the code run for 60 seconds remove this later 
bronze_query.stop()



In [0]:
# have code to wait for the write bronze_stream to await terminate before doing later code 





In [0]:
display(dbutils.fs.ls(BRONZE_DELTA)) # checking to see that the bronze_delta table exists in this path 

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# ENTER YOUR CODE HERE

# reading the delta table from BRONZE_DELTA location and then getting the count 
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)
display(bronze_df)

print("the number of tweets captured in bronze table", bronze_df.count())

# code to show the count of all the none, null, NaN values 
for c in bronze_df.columns: 
   count_of_nan = bronze_df.filter(bronze_df[c].isNull()).count()
   print("number of null values for column", c, "", count_of_nan)





There does exist nan and null values in this data frame. In order to address this, I would personally delete the entire row, because one, there are 200001 tweets so removing rows with nan/null will not be many relative to the total number of tweets.

In [0]:

# counting number of tweets by each unique user handle and sort by descending count 
unique_users = (bronze_df
            .groupBy('user') # grouping by the distinct user 
            .count()
            .orderBy(desc("count"))
            .show()
)


In [0]:
from pyspark.sql import functions as F 
# Selecting rows with mentions of at least one @

# Define the regex pattern to extract '@' symbol
regex_pattern = r".*@.*"

# Filter the DataFrame to select rows where the 'text' column matches the regex pattern
filtered_data_at_least_one_mention = bronze_df.filter(expr("text RLIKE '" + regex_pattern + "'"))

# Count the number of rows in the filtered DataFrame
at_least_one_mentions_users = filtered_data_at_least_one_mention.count()

print("Count for number of tweets with at least one mention @:", at_least_one_mentions_users )

# Counting the tweets without any mentions (@)
no_mentions_users = bronze_df.filter(~F.col("text").contains("@")).count()
print("Count for number of tweets without any '@' mention:", no_mentions_users)

In [0]:
# Plot a bar chart that shows the top 20 tweeters (users)

# first get the dataframe of the top 20 tweeters and the respective count for each 

import plotly.express as px

# Convert Spark DataFrame to Pandas DataFrame
top_twenty_pd = top_twenty.toPandas()

# Plot the bar chart using the Pandas DataFrame
fig = px.bar(top_twenty_pd, x='user', y='count')
fig.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

In [0]:
from pyspark.sql.functions import col, to_timestamp, expr

bronze_df = (bronze_df
                .withColumn('formatted_date',expr("substring(date, 5, length(date))"))
                .withColumn('timestamp', to_timestamp(col('formatted_date'), 'MMM dd HH:mm:ss z yyyy'))
                .withColumn('cleaned_text', regex_extract())

                .select('timestamp', 'mention', 'cleaned_text', 'sentiment')
)

display(bronze_df)




## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# ENTER YOUR CODE HERE
# ml flow has methods to load model out of registry so that you can operate in parallel on your cluster 

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# ENTER YOUR CODE HERE
#every time you run this, you will have a new run in ml flow tracking 

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# ENTER YOUR CODE HERE

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

# searching all the streams that are still running 
    for stream in spark.streams.active:
        print("active stream", stream)

# stopping all active streams 
stop_all_streams()






In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
elapsed_time = END_TIME - START_TIME 
print("total time elapsed:", elapsed_time)

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE